In [1]:
drive=False
if drive:
    from google.colab import drive
    drive.mount('/content/drive')

# Font Classification Project

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.config.run_functions_eagerly(True)

Num GPUs Available:  1


Working with tensorflow and local GPU (GTX 1070)

## Functions for data generation

In [3]:
import h5py
import matplotlib.pyplot as plt
import cv2
import numpy as np
import sklearn
import sklearn.model_selection

In [4]:
input_height=64
input_width=64

In [5]:
def fonts_to_output(fonts, label_encoder):
    label = label_encoder.transform(fonts)
    return tf.keras.utils.to_categorical(label, 7)

In [6]:
def crop_wrap_letter(img, bounding_box):
    src=bounding_box
    input_pts = np.float32(src)
    # y comes first in the matrix
    output_pts = np.float32([[0, 0],
                            [input_height - 1, 0],
                            [input_height - 1, input_width - 1],
                            [0, input_width - 1]])
    # Compute the perspective transform M
    M = cv2.getPerspectiveTransform(input_pts,output_pts)
    out = cv2.warpPerspective(img,
                          M,
                          (input_width, input_height),
                          flags=cv2.INTER_LINEAR)
    return out

In [7]:
def crop_wrap_image(img, charBB, fonts, label_encoder=None):
    mini_batch = []
    for i in range(fonts.shape[0]):
        bounding_box = np.dstack(charBB[:,:,i])[0]  # pairs of x,y
        crop = tf.keras.applications.resnet50.preprocess_input(crop_wrap_letter(img, bounding_box))  # Identify as 1 batch of image
        mini_batch.append(crop)
        crop = crop.reshape((1,input_height,input_width,3))  # Identify as 1 batch of image
        if not label_encoder:
            yield crop
        else:
            yield crop, fonts_to_output([fonts[i]], label_encoder)
    # mini_batch = np.asarray(mini_batch)
    # if not label_encoder:
    #     yield mini_batch
    # else:
    #     yield mini_batch, np.array(label_encoder.transform(fonts).reshape((-1,1)))

In [8]:
def crop_wrap_data(db, im_names, label_encoder=None):
    for im in im_names:
        img = db['data'][im][:]
        fonts = db['data'][im].attrs['font']
        txt = db['data'][im].attrs['txt']
        charBB = db['data'][im].attrs['charBB']
        wordBB = db['data'][im].attrs['wordBB']
        
        yield from crop_wrap_image(img, charBB, fonts, label_encoder)

In [9]:
def data_generator(*args, **kwargs):
    while True:
        yield from crop_wrap_data(*args, **kwargs)

In [10]:
def font_label_data(db, im_names, label_encoder=None):
    for im in im_names:
        font = db['data'][im].attrs['font']
        
        for f in font:
            if not label_encoder:
                yield f
            else:
                yield fonts_to_output([f], label_encoder)

In [11]:
def load_db():
    path = 'SynthText.h5' if not drive else '/content/drive/MyDrive/Fonter/SynthText.h5'
    db = h5py.File(path, 'r')
    im_names = list(db['data'].keys())
    return db, im_names

In [12]:
def data_size(db, im_names):
    size = 0
    for im in im_names:
        font = db['data'][im].attrs['font']
        size += len(font)
    return size

## Prepare the data

In [13]:
db, im_names = load_db()

In [14]:
label_encoder = sklearn.preprocessing.LabelEncoder()
label_encoder.fit(list(font_label_data(db, im_names)))
label_encoder.classes_, len(label_encoder.classes_)

(array([b'Alex Brush', b'Michroma', b'Open Sans', b'Raleway', b'Roboto',
        b'Russo One', b'Ubuntu Mono'], dtype='|S11'),
 7)

In [15]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
(mnist_train, mnist_train_labels), (mnist_test, mnist_test_labels) = mnist.load_data()
tf.keras.utils.to_categorical

<function keras.utils.np_utils.to_categorical(y, num_classes=None, dtype='float32')>

In [16]:
train_cat_labels = tf.keras.utils.to_categorical(mnist_train_labels)
test_cat_labels = tf.keras.utils.to_categorical(mnist_test_labels)

In [17]:
test_cat_labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
mnist_train_color = np.array([cv2.resize(cv2.cvtColor(x,cv2.COLOR_GRAY2RGB), (input_height,input_width)) for x in mnist_train])
mnist_test_color = np.array([cv2.resize(cv2.cvtColor(x,cv2.COLOR_GRAY2RGB),(input_height,input_width)) for x in mnist_test])

In [19]:
mnist_test_color.shape

(10000, 64, 64, 3)

In [20]:
partial_data, _ = sklearn.model_selection.train_test_split(im_names, test_size=0.99, random_state = 42)  # testing purposes

In [21]:
X_train, X_test = sklearn.model_selection.train_test_split(partial_data, test_size=0.2, random_state = 42)

In [22]:
X_train, X_val = sklearn.model_selection.train_test_split(X_train, test_size=0.1, random_state = 42)

In [23]:
train_data_and_labels = data_generator(db, X_train, label_encoder=label_encoder)
val_data_and_labels = data_generator(db, X_val, label_encoder=label_encoder)
test_data = data_generator(db, X_test)
test_labels = label_encoder.transform(list(font_label_data(db, X_test)))

In [24]:
# size = data_size(db, X_train)
# for i, (img, label) in enumerate(train_data_and_labels):
#     if i>=size*3:
#         break
#     if i%size!=0:
#         continue
#     plt.figure()
#     plt.imshow(img.reshape((input_height,input_width,3)))
#     plt.title(label[0])

## Define a model

In [25]:
resnet = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=(input_height,input_width,3))

In [26]:
# Freeze the layers which you don't want to train. Here I am freezing the all layers.
resnet.trainable = False
resnet.training = False

In [27]:
# inputs = tf.keras.Input(shape=(input_height, input_width, 3))
# x = resnet(inputs, training=False)
# # x = tf.keras.layers.GlobalAveragePooling2D()(x)
# # x = tf.keras.layers.Dropout(0.2)(x)
# outputs = tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')(x)
# fonter = tf.keras.Model(inputs, outputs)

# # # ###could be defined as Sequential I think..

In [28]:
fonter = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(input_height,input_width,3)),
    resnet,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax"),
])

In [29]:
base_learning_rate = 0.001
opt = tf.keras.optimizers.Nadam(learning_rate=base_learning_rate)
fonter.compile(optimizer=opt,
              loss = "categorical_crossentropy",
              metrics=['accuracy'])

In [30]:
fonter.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 resnet50 (Functional)       (None, 2, 2, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 512)               2

## Train Model

In [31]:
initial_epochs = 10
loss0, accuracy0 = fonter.evaluate(mnist_test_color, test_cat_labels)

313/313 [==============================] - 14s 29ms/step - loss: 2.4092 - accuracy: 0.0995


In [32]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 2.41
initial accuracy: 0.10


In [33]:
history = fonter.fit(mnist_train_color, train_cat_labels, batch_size=32,
                     epochs=initial_epochs,
                     validation_data=(mnist_test_color, test_cat_labels),
                    #validation_data=val_data_and_labels,
                    #max_queue_size=1,
                    #steps_per_epoch=data_size(db, X_train),
                    #validation_steps=data_size(db, X_val)
                    )

Epoch 1/10
1875/1875 [==============================] - 74s 38ms/step - loss: 0.2356 - accuracy: 0.9247 - val_loss: 0.4645 - val_accuracy: 0.8639
Epoch 2/10
1875/1875 [==============================] - 71s 38ms/step - loss: 0.1512 - accuracy: 0.9513 - val_loss: 0.3244 - val_accuracy: 0.8983
Epoch 3/10
1875/1875 [==============================] - 71s 38ms/step - loss: 0.1300 - accuracy: 0.9574 - val_loss: 0.3231 - val_accuracy: 0.9057
Epoch 4/10
 101/1875 [>.............................] - ETA: 59s - loss: 0.0922 - accuracy: 0.9734

KeyboardInterrupt: 

In [ ]:
# history = fonter.fit(train_data_and_labels,
#                     epochs=initial_epochs,
#                     validation_data=val_data_and_labels,
#                     max_queue_size=1,
#                     steps_per_epoch=data_size(db, X_train),
#                     validation_steps=data_size(db, X_val)
#                     )

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
imgs.shape

In [ ]:
fonter.evaluate(imgs, labels, batch_size=1)

In [ ]:
predictions = fonter.predict(imgs)

In [ ]:
np.argmax(predictions, axis=1)

In [ ]:
np.argmax(labels, axis=1)

In [ ]:
for i, (img, label) in enumerate(val_data_and_labels):
    if i==data_size(db, X_val)*2:
        break
    plt.figure()
    plt.imshow(img.reshape((input_height,input_width,3)))
    plt.title(label[0])

In [ ]:
X_train

In [ ]:
im_names.index(X_val[0])

In [ ]:
im_names.index(X_train[-1])

In [ ]:
im_names.index(X_test[0])